In [12]:
import pandas as pd
import pysam
import re
import argparse

In [13]:
def get_pairs(regions, chrom1, chrom2):
    """
    Fetch all paired reads, including those with split alignments

    Parameters:
    - regions: the two SV regions to collect reads from
    - chrom1: chromosome # of first region
    - chrom2: chromosome # of second region

    Returns:
    - split_alignments: contains all sets of paired reads with split alignments
    - paired_alignments: contains all sets of paired reads without split alignments
    """
    split_alignments = []
    nonsplit_alignments = []

    # Iterate over first breakpoint end
    for i, region in enumerate(regions):
        for aln in region:
            # If we've seen this read before, skip it
            if any(aln in tup for tup in split_alignments) or any(aln in tup for tup in nonsplit_alignments):
                # print("Duplicate found")
                continue

            mate = samfile.mate(aln)

            if aln.is_supplementary or aln.has_tag("SA"):
                sa_tag = (aln.get_tag("SA")).split(";")[:-1]
                sa_tag = sa_tag[0]
                sup_name, sup_pos, sup_strand, sup_cigar, sup_mapq, sup_tlen = sa_tag.split(",")
                sup_pos = int(sup_pos)
                sup_tlen = int(sup_tlen)

                found_split2 = False
                for sup_read in samfile.fetch(sup_name, sup_pos, sup_pos + 1):
                    if (sup_read.query_name == aln.query_name and sup_read.reference_name == sup_name and (sup_read.reference_start + 1) == sup_pos and re.sub(r'[SH]', '', sup_read.cigarstring) == re.sub(r'[SH]', '', sup_cigar)):
                        non_sup = aln if not aln.is_supplementary else sup_read
                        sup = aln if aln.is_supplementary else sup_read
                        read1 = non_sup if non_sup.is_read1 else mate
                        read2 = non_sup if non_sup.is_read2 else mate
                        split_alignments.append((read1, sup, read2))
                        found_split2 = True
                        break
                if not found_split2:
                    # print(sup_name, sup_pos, sup_cigar, sup_tlen)
                    raise NameError("split mate not found")

            elif mate.is_supplementary or mate.has_tag("SA"):
                sa_tag = (mate.get_tag("SA")).split(";")[:-1]
                sa_tag = sa_tag[0]
                sup_name, sup_pos, sup_strand, sup_cigar, sup_mapq, sup_tlen = sa_tag.split(",")
                sup_pos = int(sup_pos)
                sup_tlen = int(sup_tlen)

                found_split2 = False
                for sup_read in samfile.fetch(sup_name, sup_pos, sup_pos + 1):
                    if (sup_read.query_name == mate.query_name and sup_read.reference_name == sup_name and (sup_read.reference_start + 1) == sup_pos and re.sub(r'[SH]', '', sup_read.cigarstring) == re.sub(r'[SH]', '', sup_cigar)):
                        non_sup = mate if not mate.is_supplementary else sup_read
                        sup = mate if mate.is_supplementary else sup_read
                        read1 = non_sup if non_sup.is_read1 else aln
                        read2 = non_sup if non_sup.is_read2 else aln
                        split_alignments.append((read1, sup, read2))
                        found_split2 = True
                        break
                if not found_split2:
                    # print(sup_name, sup_pos, sup_cigar, sup_tlen)
                    raise NameError("split mate not found")
                
            elif not aln.is_proper_pair and aln.next_reference_name == (chrom2 if i == 0 else chrom1):
                read1 = aln if aln.is_read1 else mate
                read2 = aln if aln.is_read2 else mate
                nonsplit_alignments.append((read1, read2))
            
            elif aln.is_proper_pair:
                read1 = aln if aln.is_read1 else mate
                read2 = aln if aln.is_read2 else mate
                nonsplit_alignments.append((read1, read2))
    
    return split_alignments, nonsplit_alignments

In [14]:
def fetch_alignments(bamfile, chrom1, pos1, chrom2, pos2, sv_type, read_support, features, orientation, hom_len, hom, region_size=150, mapq_threshold=15):
    """
    Fetch reads from a BAM file that have split alignments 
    between two genomic coordinates as well as all paired-end reads.

    Parameters:
    - bamfile: str, path to the BAM file.
    - chrom1: str, chromosome for the first position.
    - pos1: int, first position in the genome.
    - chrom2: str, chromosome for the second position.
    - pos2: int, second position in the genome.
    - region_size: int, the size of the region around the positions to fetch alignments from.
    - mapq_threshold: int, cutoff for collecting reads above certain mapping quality.

    Returns:
    - split_df: DataFrame, details of split alignments.
    - paired_df: DataFrame, details of discordant alignments.
    """
    # Open the BAM file
    samfile = pysam.AlignmentFile(bamfile, "rb")

    # Fetch alignments region_size away from both positions
    region1 = samfile.fetch(chrom1, pos1 - region_size, pos1 + region_size + 1)
    region2 = samfile.fetch(chrom2, pos2 - region_size, pos2 + region_size + 1)

    split_alignments, nonsplit_alignments = get_pairs([region1, region2], chrom1, chrom2)
    # Collect details of split alignments
    split_alignment_details = []
    for pair in split_alignments:
        read1_1, read1_2, read2 = pair

        # Filter out low quality reads
        if  ((read1_1.mapping_quality > mapq_threshold and read1_2.mapping_quality > mapq_threshold) or read2.mapping_quality > mapq_threshold) or ((read1_1.is_mapped and read1_2.is_mapped) or read2.is_mapped):
            for read in pair:
                split_alignment_details.append({
                    "break_chrom1": chrom1,
                    "break_pos1": pos1,
                    "break_chrom2": chrom2,
                    "break_pos2": pos2,
                    "break_sv_type": sv_type,
                    "break_read_support": read_support,
                    "break_features": features,
                    "break_orientation": orientation,
                    "AA_homology_len": hom_len,
                    "AA_homology_seq": hom,
                    "query_name": read.query_name,
                    "query_short": read.query_name.split(':')[-1],
                    "split": read.has_tag("SA"),
                    "proper_pair": "Concordant" if read.is_proper_pair else "Discordant",
                    "read_num": "1" if read.is_read1 else "2",
                    "query_chrom": read.reference_name,
                    "query_pos": read.reference_start + 1,
                    "query_end": read.reference_end + 1,
                    "query_orientation": "+" if read.is_forward else "-",
                    "query_cigar": read.cigarstring,
                    "query_aln_full": read.query_sequence,
                    "query_aln_sub": read.query_alignment_sequence
                })

    # Collect details of nonsplit alignments
    nonsplit_alignment_details = []
    for pair in nonsplit_alignments:
        read1, read2 = pair
        # Filter out low quality reads
        if  (read1.mapping_quality > mapq_threshold or read2.mapping_quality > mapq_threshold) or (read1.is_mapped or read2.is_mapped):
            for read in pair:
                nonsplit_alignment_details.append({
                    "break_chrom1": chrom1,
                    "break_pos1": pos1,
                    "break_chrom2": chrom2,
                    "break_pos2": pos2,
                    "break_sv_type": sv_type,
                    "break_read_support": read_support,
                    "break_features": features,
                    "break_orientation": orientation,
                    "AA_homology_len": hom_len,
                    "AA_homology_seq": hom,
                    "query_name": read.query_name,
                    "query_short": read.query_name.split(':')[-1],
                    "split": read.has_tag("SA"),
                    "proper_pair": "Concordant" if read1.is_proper_pair else "Discordant",
                    "read_num": "1" if read.is_read1 else "2",
                    "query_chrom": read.reference_name,
                    "query_pos": read.reference_start + 1,
                    "query_end": read.reference_end + 1,
                    "query_orientation": "+" if read.is_forward else "-",
                    "query_cigar": read.cigarstring,
                    "query_aln_full": read.query_sequence,
                    "query_aln_sub": read.query_alignment_sequence
                })
        
    samfile.close()

    # Convert details to pandas DataFrames
    split_df = pd.DataFrame(split_alignment_details)
    nonsplit_df = pd.DataFrame(nonsplit_alignment_details)
    # paired_df = pd.DataFrame(paired_alignment_details)

    return split_df, nonsplit_df

In [16]:
bamfile = "/home/asureshkumar/Documents/research/ecDNA-Alignment/K562/K562_hg19_cnvkit.cs.rmdup.bam"
sumfile = "/home/asureshkumar/Documents/research/ecDNA-Alignment/K562/K562_summaries.tsv"
samfile = pysam.AlignmentFile(bamfile, "rb")
df = pd.read_csv(sumfile, sep="\t")
# Define output table
output = pd.DataFrame()
num_split = 0
num_paired = 0
df = df.drop(["pos1_flanking_coordinate", "pos2_flanking_coordinate", "sample"], axis=1)
display(df)

for index, row in df.iterrows():

    # Fetch the DataFrames for split alignments and paired alignments
    split_df, nonsplit_df = fetch_alignments(bamfile, *row, 150)
    display(split_df)
    display(nonsplit_df)
    num_split += len(split_df)/3
    num_paired += (2 * len(split_df) / 3) + len(nonsplit_df)
    # num_paired += len(paired_df)
    

    print(f"Reads at junction defined by {row.chrom1} {row.pos1} and {row.chrom2} {row.pos2}:")
    print(f"Number split pairs: {len(split_df)/3}")
    print(f"Number nonsplit pairs: {len(nonsplit_df)/2}")
    print(f"Number paired: {(2 * len(split_df) / 3) + len(nonsplit_df)}")
    output = pd.concat([split_df, nonsplit_df], ignore_index=True)
    break

output.to_csv('alignments.tsv', sep='\t', index=False)

print("Total Number of split reads:", num_split)
print("Total Number of paired reads:", num_paired)

,chrom1,pos1,chrom2,pos2,sv_type,read_support,features,orientation,homology_length,homology_sequence
0,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG
1,chr9,134155523,chr13,108661410,interchromosomal,6,unknown,++,2.0,AT
2,chr13,81088032,chr13,81469934,inversion,4,ecDNA,--,3.0,AAC
3,chr13,81109413,chr13,90438706,deletion-like,3,ecDNA,+-,3.0,TGG
4,chr13,81470035,chr13,81470310,foldback,3,ecDNA,++,6.0,ATAAAA
5,chr13,90862773,chr13,90864806,deletion-like,3,ecDNA,+-,NaN,NaN
6,chr13,92474010,chr13,92475757,foldback,3,NaN,++,2.0,AA
7,chr13,94023407,chr13,108500972,deletion-like,3,unknown,+-,NaN,NaN
8,chr22,18953122,chr22,22935373,inversion,5,unknown,--,NaN,NaN
9,chr22,20950573,chr22,20953841,deletion-like,4,unknown,+-,NaN,NaN


,break_chrom1,break_pos1,break_chrom2,break_pos2,break_sv_type,break_read_support,break_features,break_orientation,AA_homology_len,AA_homology_seq,...,split,proper_pair,read_num,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_full,query_aln_sub
0,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,True,Concordant,1,chr9,133607147,133607213,+,34S66M,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTA...,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAG...
1,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,True,Concordant,1,chr22,23632704,23632743,+,39M61H,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG
2,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,2,chr9,133607344,133607444,-,100M,TGTTTCAAATTTTCAACTTTCAAAATTCCTGCTGCTTCTTTCCAAT...,TGTTTCAAATTTTCAACTTTCAAAATTCCTGCTGCTTCTTTCCAAT...
3,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,True,Concordant,1,chr22,23632682,23632743,-,61M39S,GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCT...,GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCT...
4,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,True,Concordant,1,chr9,133607147,133607191,-,56H44M,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA
5,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,2,chr22,23632592,23632692,+,100M,AGAGTTCAAGTAAGTACTGGTTTGGGGAGGAGGGTTGCAGCGGCCG...,AGAGTTCAAGTAAGTACTGGTTTGGGGAGGAGGGTTGCAGCGGCCG...
6,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,1,chr22,23632535,23632635,+,100M,CTCCGGGGCTCTATGGGTTTCTGAATGTCATCGTCCACTCAGCCAC...,CTCCGGGGCTCTATGGGTTTCTGAATGTCATCGTCCACTCAGCCAC...
7,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,True,Concordant,2,chr9,133607147,133607178,-,69H31M,GAGTGGGTTTTATCAGCTTCCATACCCAAAC,GAGTGGGTTTTATCAGCTTCCATACCCAAAC
8,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,True,Concordant,2,chr22,23632669,23632743,-,74M26S,GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGA...,GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGA...
9,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,True,Discordant,1,chr9,133607147,133607218,-,29S71M,ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGC...,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAG...


,break_chrom1,break_pos1,break_chrom2,break_pos2,break_sv_type,break_read_support,break_features,break_orientation,AA_homology_len,AA_homology_seq,...,split,proper_pair,read_num,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_full,query_aln_sub
0,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,1,chr9,133606903,133607003,+,100M,CCCTCATATTGACCTAGCTGTTTAATTGAAAAGAACATCTGCTGCT...,CCCTCATATTGACCTAGCTGTTTAATTGAAAAGAACATCTGCTGCT...
1,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,2,chr9,133607120,133607220,-,100M,CTCCTTTGTATTTCCATATACATTTTAGAGTGGGTTTTATCAGCTT...,CTCCTTTGTATTTCCATATACATTTTAGAGTGGGTTTTATCAGCTT...
2,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,1,chr9,133606645,133606745,+,100M,TGGCCTCCCAAAGTGCTGGGGTTACAGACGTGAGCCACTGCGTCCG...,TGGCCTCCCAAAGTGCTGGGGTTACAGACGTGAGCCACTGCGTCCG...
3,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,2,chr9,133607005,133607105,-,100M,TTTGTTTGTCTTTCCTTGCACCAGTATTACAATGTATTACTGTAGC...,TTTGTTTGTCTTTCCTTGCACCAGTATTACAATGTATTACTGTAGC...
4,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,1,chr9,133607067,133607167,-,100M,ATGCCTGGTAGAATAAGGCCTCCTGTTTTGTTCTTCAATATTGTCT...,ATGCCTGGTAGAATAAGGCCTCCTGTTTTGTTCTTCAATATTGTCT...
5,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,2,chr9,133606727,133606827,+,100M,ACCTTAATTTTGACTTAGTCTAATTTAATCTTTTATGGTTAGTACT...,ACCTTAATTTTGACTTAGTCTAATTTAATCTTTTATGGTTAGTACT...
6,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,1,chr9,133607147,133607226,+,21S79M,CCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATAC...,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAG...
7,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,2,chr9,133607451,133607551,-,100M,CCCTTGATAGTGGAGACCATGTTTATACATTTTCATGGATCGAACA...,CCCTTGATAGTGGAGACCATGTTTATACATTTTCATGGATCGAACA...
8,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,1,chr9,133607186,133607286,-,100M,CCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGGTTTCTT...,CCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGGTTTCTT...
9,chr9,133607146,chr22,23632741,interchromosomal,7,unknown,-+,5.0,GAGTG,...,False,Concordant,2,chr9,133607147,133607224,+,23S77M,CACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCAT...,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAG...


Reads at junction defined by chr9 133607146 and chr22 23632741:
Number split pairs: 5.0
Number nonsplit pairs: 24.0
Number paired: 58.0
Total Number of split reads: 5.0
Total Number of paired reads: 58.0
